In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
import torch.optim as optim

In [4]:
import torch.nn.functional as F

In [5]:
import torchvision

In [6]:
import torchvision.transforms as transforms

In [7]:
import numpy as np
import pandas as pd

In [10]:
train_set = torchvision.datasets.MNIST(
     root='C:/Users/tpath/Desktop/ECEfiles'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()])
)

Extracting C:/Users/tpath/Desktop/ECEfiles\MNIST\raw\train-images-idx3-ubyte.gz to C:/Users/tpath/Desktop/ECEfiles\MNIST\raw


Extracting C:/Users/tpath/Desktop/ECEfiles\MNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/tpath/Desktop/ECEfiles\MNIST\raw


Extracting C:/Users/tpath/Desktop/ECEfiles\MNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/tpath/Desktop/ECEfiles\MNIST\raw


Extracting C:/Users/tpath/Desktop/ECEfiles\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/tpath/Desktop/ECEfiles\MNIST\raw
Processing...
Done!


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [31]:
train_loader = torch.utils.data.DataLoader(train_set ,batch_size=500 ,shuffle=True )

In [32]:
train_set.targets.bincount()

tensor([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949])

In [33]:
len(train_set[0])

2

In [41]:
print(type(train_set[0][0]))
print(type(train_set[0][1]))

<class 'torch.Tensor'>
<class 'int'>


In [42]:
train_set[0][0].shape

torch.Size([1, 28, 28])

In [43]:
torch.tensor(train_set[0][1]).shape

torch.Size([])

In [49]:
sample = next(iter(train_loader))
sample[0].shape

torch.Size([500, 1, 28, 28])

In [56]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        return t

In [57]:
network = Network()            

In [58]:
network

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [59]:
network.conv1

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [60]:
network.conv1.weight

Parameter containing:
tensor([[[[-0.0276, -0.1655, -0.1196, -0.1209, -0.1680],
          [-0.0536, -0.1139,  0.1004,  0.1191,  0.1779],
          [-0.0793,  0.0141, -0.1144,  0.0760, -0.1506],
          [ 0.0554, -0.0796, -0.0030,  0.1510,  0.1655],
          [-0.0508,  0.1176, -0.1932,  0.0658, -0.0407]]],


        [[[-0.0197,  0.0519,  0.0173,  0.1306, -0.0550],
          [-0.1886,  0.0112, -0.1975, -0.1647,  0.0789],
          [-0.0186, -0.0250,  0.0682, -0.0700, -0.0302],
          [ 0.1397,  0.1027,  0.1053,  0.1461, -0.1387],
          [-0.1482, -0.0218,  0.1427,  0.1694, -0.0029]]],


        [[[-0.1344,  0.0030, -0.0917, -0.1001, -0.1558],
          [-0.0339,  0.1667, -0.0806, -0.0230, -0.1137],
          [ 0.0146, -0.1629, -0.1047,  0.1715, -0.0550],
          [-0.0936,  0.0529,  0.0509, -0.1381,  0.0816],
          [ 0.0222, -0.1644, -0.1326,  0.1165,  0.0672]]],


        [[[ 0.0579, -0.0313, -0.0827,  0.0443,  0.0143],
          [-0.0978, -0.1965, -0.0507,  0.1710, -0.1190

In [62]:
batch = next(iter(train_loader))
images, labels = batch

In [63]:
images.shape

torch.Size([500, 1, 28, 28])

In [64]:
labels.shape

torch.Size([500])

In [65]:
preds = network(images)

In [66]:
preds.shape

torch.Size([500, 10])

In [69]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [70]:
get_num_correct(preds, labels)

50

In [71]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
batch = next(iter(train_loader)) # Getting a batch
images, labels = batch

In [73]:
preds = network(images)
loss = F.cross_entropy(preds, labels) # Calculating the loss
print(loss.item())
get_num_correct(preds, labels)

2.3141303062438965


11

In [74]:
loss.backward()

In [75]:
optimizer = optim.Adam(network.parameters(), lr=0.01)
optimizer.step() # Updating the weights

In [76]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 56375 loss: 121.00830533826957
epoch 1 total_correct: 58622 loss: 48.742995633685496
epoch 2 total_correct: 58849 loss: 39.844745602888
epoch 3 total_correct: 58884 loss: 38.61416319417913
epoch 4 total_correct: 58912 loss: 39.43419021804948
epoch 5 total_correct: 59016 loss: 36.26349980998202
epoch 6 total_correct: 59034 loss: 36.7033335331314
epoch 7 total_correct: 59110 loss: 34.507049874702716
epoch 8 total_correct: 59142 loss: 33.936428003085894
epoch 9 total_correct: 59146 loss: 35.19185064790986
